# NLP - HW2

## Punto No. 2: Naive Bayes (NB) & Logistic Regression (LR)

### For the 20N dataset compare two classifiers NB and LR to identify the 20 different newsgroups.

##### Create your own processing pipeline for the task and justify it.

In [4]:
# import libraries

import pyspark.sql.functions as F


##### Divide the dataset into training (60%), development (10%) and test (30%)

##### Train NB and LR using the following representations:

###### BOW

###### Boolean BOW

###### Personalized representation. You as a designer must define the select set of characteristics. Explain your feature selection strategy in detail.

#### Compare the results of NB and LR using 10-fold cross validation:

##### Use for cross validation: training+development sets.

##### Do a search for LR hyperparameters

##### Report the average results, precision, and recall by class.

#### Evaluate models using the test set:

##### Precision, recall, F1, accuracy with the micro and macro averaging strategies.

#### Compare the results in terms of:

##### NB vs LR

##### Features

##### Dataset and clasess distribution.